# Opnames per leeftijds plot

Deze data is vrij schaars, qua updates en qua details. Dus roeien met de riemen die we hebben.

## Python voorbereiden

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.dates as mdates
from datetime import date,datetime,timedelta
import matplotlib.pyplot as plt

In [ ]:
sns.set_style("whitegrid")
sns.set_context("talk", font_scale=1, rc={"lines.linewidth": 3, "font.size":10,"axes.titlesize":24,"axes.labelsize":18})
sns.set_palette(sns.color_palette(['#CC6677', '#332288', '#DDCC77', '#117733', '#88CCEE', \
                    '#882255', '#44AA99', '#999933', '#AA4499', '#DDDDDD', \
                    '#000000']))



def tuftefy(ax):
    """Remove spines and tick position markers to reduce ink."""
    # 
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_visible(True)
    ax.spines["bottom"].set_color('grey')

    ax.grid(color="w", alpha=0.5)
    ax.get_yaxis().grid(True)
    ax.get_xaxis().grid(False)

def fancy_titles(t1, t2, ax=None):
  if ax:
    ax.set_title(t1, loc='left', fontsize=18)
    ax.set_title(t2, loc='right', fontsize=13, color='grey')
  else:
    plt.title(t1, loc='left', fontsize=18)
    plt.title(t2, loc='right', fontsize=13, color='grey')



# Date importeren en verwerken

In [ ]:

opnames = pd.read_csv('https://data.rivm.nl/covid-19/COVID-19_ziekenhuis_ic_opnames_per_leeftijdsgroep.csv', sep=';', parse_dates=["Date_of_statistics_week_start"])
opnames

In [ ]:
opnames2 = opnames.sort_values(by=["Age_group", "Date_of_statistics_week_start"])
#.rolling(window=7).mean().reset_index()
opnames2['Hospital_admission_notification_7d'] = opnames2.Hospital_admission_notification.rolling(window=5).mean()
opnames2 = opnames2[opnames2.Date_of_statistics_week_start > '2020-05-01']
opnames2


In [ ]:
opnames_nice = pd.read_csv("https://github.com/mzelst/covid-19/raw/master/data-nice/age/leeftijdsverdeling_datum_Klinisch_IC_long.csv", parse_dates=["Datum"])
opnames_nice = opnames_nice[opnames_nice.Type == "Klinisch"].sort_values("Datum")
opnames_nice.drop(columns=['Type'], inplace=True)

ic_nice = pd.read_csv("https://github.com/mzelst/covid-19/raw/master/data-nice/age/leeftijdsverdeling_datum_Klinisch_IC_long.csv", parse_dates=["Datum"])
ic_nice = ic_nice[ic_nice.Type != "Klinisch"].sort_values("Datum")
ic_nice.drop(columns=['Type'], inplace=True)

opnames_nice7 = opnames_nice.set_index('Datum').diff().rolling(window=14).mean()
opnames_nice7 = opnames_nice7.reset_index()
display(opnames_nice7)

opnames_nice = opnames_nice.set_index('Datum').diff().rolling(window=35).mean()
opnames_nice = opnames_nice.reset_index()

ic_nice14 = ic_nice.set_index('Datum').diff().rolling(window=14).mean()
ic_nice14 = ic_nice14.reset_index()
display(ic_nice14)

In [ ]:
groups = pd.DataFrame(np.array([
    ["0-14", "0-14"], 
    ["15-29", "15-19"],
    ["15-29", "20-24"],
    ["15-29", "25-29"],
    ["30-44", "30-34"],
    ["30-44", "35-39"],
    ["30-44", "40-44"],
    ["45-59", "45-49"],
    ["45-59", "50-54"],
    ["45-59", "55-49"],
    ["60-74", "60-64"],
    ["60-74", "65-69"],
    ["60-74", "70-74"],
    ["75+", "75-79"],
    ["75+", "80-84"],
    ["75+", "85-89"],
    ["75+", "90+"],
    ["Onbekend", "Unknown"]
]), columns=["Group", "Age_group"])
merged_opnames = opnames2.set_index('Age_group').join(groups.set_index('Age_group')).reset_index()
merged_opnames = merged_opnames.groupby(by=['Date_of_statistics_week_start','Group']).sum().reset_index()
merged_opnames

In [ ]:

groups = pd.DataFrame(np.array([
    ["<30", "0-14"], 
    ["<30", "15-19"],
    ["<30", "20-24"],
    ["<30", "25-29"],
    ["30+", "30-34"],
    ["30+", "35-39"],
    ["30+", "40-44"],
    ["30+", "45-49"],
    ["30+", "50-54"],
    ["30+", "55-49"],
    ["30+", "60-64"],
    ["30+", "65-69"],
    ["30+", "70-74"],
    ["30+", "75-79"],
    ["30+", "80-84"],
    ["30+", "85-89"],
    ["30+", "90+"],
    ["30+", "Unknown"]
]), columns=["Group", "Age_group"])
merged_opnames2 = opnames2.set_index('Age_group').join(groups.set_index('Age_group')).reset_index()
merged_opnames2 = merged_opnames2.groupby(by=['Date_of_statistics_week_start','Group']).sum().reset_index()
merged_opnames2

In [ ]:
opnames_nice['<30'] = opnames_nice['<20'] + opnames_nice['20 - 24'] + opnames_nice['25 - 29']
opnames_nice['30 - 60'] = opnames_nice['30 - 34'] + opnames_nice['35 - 39'] + \
    opnames_nice['40 - 44'] + opnames_nice['45 - 49'] + \
    opnames_nice['50 - 54'] + opnames_nice['55 - 59']
    
opnames_nice['60+'] = opnames_nice['60 - 64'] + opnames_nice['65 - 69'] + \
    opnames_nice['70 - 74'] + opnames_nice['75 - 79'] + \
    opnames_nice['80 - 84'] + opnames_nice['85 - 89'] + \
    opnames_nice['>90']

opnames_nice['30+'] = opnames_nice['30 - 60'] + opnames_nice['60+']

opnames_nice7['<30'] = opnames_nice7['<20'] + opnames_nice7['20 - 24'] + opnames_nice7['25 - 29']
opnames_nice7['30 - 60'] = opnames_nice7['30 - 34'] + opnames_nice7['35 - 39'] + \
    opnames_nice7['40 - 44'] + opnames_nice7['45 - 49'] + \
    opnames_nice7['50 - 54'] + opnames_nice7['55 - 59']
    
opnames_nice7['60+'] = opnames_nice7['60 - 64'] + opnames_nice7['65 - 69'] + \
    opnames_nice7['70 - 74'] + opnames_nice7['75 - 79'] + \
    opnames_nice7['80 - 84'] + opnames_nice7['85 - 89'] + \
    opnames_nice7['>90']

opnames_nice7['30+'] = opnames_nice7['30 - 60'] + opnames_nice7['60+']

opnames_nice7['0-9'] = opnames_nice7['<20']
opnames_nice7['10-19'] = opnames_nice7['<20']
opnames_nice7['20-29'] = opnames_nice7['20 - 24'] + opnames_nice7['25 - 29']
opnames_nice7['30-39'] = opnames_nice7['30 - 34'] + opnames_nice7['35 - 39']
opnames_nice7['40-49'] = opnames_nice7['40 - 44'] + opnames_nice7['45 - 49']
opnames_nice7['50-59'] = opnames_nice7['50 - 54'] + opnames_nice7['55 - 59']
opnames_nice7['60-69'] = opnames_nice7['60 - 64'] + opnames_nice7['65 - 69']
opnames_nice7['70-79'] = opnames_nice7['70 - 74'] + opnames_nice7['75 - 79']
opnames_nice7['80-89'] = opnames_nice7['80 - 84'] + opnames_nice7['85 - 89']
opnames_nice7['90+'] = opnames_nice7['>90']

ic_nice14['<30'] = ic_nice14['<20'] + ic_nice14['20 - 24'] + ic_nice14['25 - 29']
ic_nice14['30 - 60'] = ic_nice14['30 - 34'] + ic_nice14['35 - 39'] + \
    ic_nice14['40 - 44'] + ic_nice14['45 - 49'] + \
    ic_nice14['50 - 54'] + ic_nice14['55 - 59']
    
ic_nice14['60+'] = ic_nice14['60 - 64'] + ic_nice14['65 - 69'] + \
    ic_nice14['70 - 74'] + ic_nice14['75 - 79'] + \
    ic_nice14['80 - 84'] + ic_nice14['85 - 89'] + \
    ic_nice14['>90']

ic_nice14['30+'] = ic_nice14['30 - 60'] + ic_nice14['60+']

ic_nice14['0-9'] = ic_nice14['<20']
ic_nice14['10-19'] = ic_nice14['<20']
ic_nice14['20-29'] = ic_nice14['20 - 24'] + ic_nice14['25 - 29']
ic_nice14['30-39'] = ic_nice14['30 - 34'] + ic_nice14['35 - 39']
ic_nice14['40-49'] = ic_nice14['40 - 44'] + ic_nice14['45 - 49']
ic_nice14['50-59'] = ic_nice14['50 - 54'] + ic_nice14['55 - 59']
ic_nice14['60-69'] = ic_nice14['60 - 64'] + ic_nice14['65 - 69']
ic_nice14['70-79'] = ic_nice14['70 - 74'] + ic_nice14['75 - 79']
ic_nice14['80-89'] = ic_nice14['80 - 84'] + ic_nice14['85 - 89']
ic_nice14['90+'] = ic_nice14['>90']

opnames_nice7


In [ ]:

ageGroupStats = pd.read_csv( "data/rivm-cases-per-ggd-per-age-group.csv", parse_dates=["Date"])
dateOfStats = ageGroupStats.Date.max()
ageGroupStats = ageGroupStats[ageGroupStats.Date <= (dateOfStats - timedelta(days=3))].copy() # remove last 3 noisy days
print(f"GGD data import: {dateOfStats}")
ageGroupStats

In [ ]:
sum7d_growth = ageGroupStats.copy().groupby(by=["Age_group", 'Date']).sum().reset_index().groupby("Age_group").rolling(on="Date", window=7).sum().reset_index()
sum7d_growth.drop(['level_1', 'Positive_cases_per100k'], axis=1, inplace=True)
sum7d_growth = sum7d_growth.pivot(index='Date', columns='Age_group', values='Positive_cases').reset_index()
sum7d_growth['<30'] = sum7d_growth['0-9'] + sum7d_growth['10-19'] + sum7d_growth['20-29']
sum7d_growth['30 - 60'] = sum7d_growth['30-39'] + sum7d_growth['40-49'] + sum7d_growth['50-59']
sum7d_growth['60+'] = sum7d_growth['60-69'] + sum7d_growth['70-79'] + sum7d_growth['80-89'] + sum7d_growth['90+']

sum7d_growth['30+'] = sum7d_growth['30 - 60'] + sum7d_growth['60+']
sum7d_growth = sum7d_growth.set_index('Date').pct_change().replace([np.inf, -np.inf], 0).fillna(0).rolling(window=21).mean() * 100
sum7d_growth = sum7d_growth.reset_index()
sum7d_growth

In [ ]:
sum7d_growth_long = pd.melt(sum7d_growth, id_vars='Date', value_name='PositieveTestGrowth')
old_date = opnames_nice7.Datum
opnames_nice7_growth = opnames_nice7.drop(['<20', '20 - 24', '25 - 29', '30 - 34', '35 - 39', '40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64', '65 - 69', '70 - 74', '75 - 79', '80 - 84', '85 - 89', '>90'], axis=1).set_index('Datum').pct_change().replace([np.inf, -np.inf], 0).fillna(0).rolling(window=21).mean() * 100
opnames_nice7_growth = opnames_nice7_growth.reset_index()
opnames_nice7_growth.rename(columns = {'Datum':'Date'}, inplace = True)

opnames_nice7_growth_long = pd.melt(opnames_nice7_growth, id_vars='Date', var_name='Age_group', value_name='OpnameGrowth')

ic_nice14_growth = ic_nice14.drop(['<20', '20 - 24', '25 - 29', '30 - 34', '35 - 39', '40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64', '65 - 69', '70 - 74', '75 - 79', '80 - 84', '85 - 89', '>90'], axis=1).set_index('Datum').pct_change().replace([np.inf, -np.inf], 0).fillna(0).rolling(window=21).mean() * 100
ic_nice14_growth = ic_nice14_growth.reset_index()
ic_nice14_growth.rename(columns = {'Datum':'Date'}, inplace = True)

ic_nice14_growth_long = pd.melt(ic_nice14_growth, id_vars='Date', var_name='Age_group', value_name='ICGrowth')

opnames_cases_joined = sum7d_growth_long.join(opnames_nice7_growth_long.set_index(['Date', 'Age_group']), on=['Date','Age_group'], how='right').reset_index()
opnames_cases_joined = ic_nice14_growth_long.join(opnames_cases_joined.set_index(['Date', 'Age_group']), on=['Date','Age_group'], how='right').reset_index()
opnames_cases_joined


## Plots RIVM data

In [ ]:
fig = plt.figure()
fig.set_size_inches(8, 6)

ax = sns.lineplot(data=opnames2[opnames2.Age_group.isin(["0-14", "15-19", "20-24"])], x="Date_of_statistics_week_start", y="Hospital_admission_notification_7d", hue="Age_group")
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
ax.set_xlabel('')
ax.set_ylabel('Opnames')
ax.set_title('Voortschrijdend gemiddelde (35d) van opnames <25', pad=30)
plt.title("bron: RIVM update " + opnames2.Date_of_statistics_week_start.max().strftime('%Y-%m-%d'), loc='right', fontsize=12, color='grey', y=0.0)
tuftefy(ax)

In [ ]:
fig = plt.figure()
fig.set_size_inches(8, 6)

ax = sns.lineplot(data=merged_opnames, x="Date_of_statistics_week_start", y="Hospital_admission_notification_7d", hue="Group")
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
ax.set_xlabel('')
ax.set_ylabel('Opnames')
ax.set_title('Voortschrijdend gemiddelde (35d) van opnames', pad=30)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.title("bron: RIVM update " + opnames2.Date_of_statistics_week_start.max().strftime('%Y-%m-%d'), loc='right', fontsize=12, color='grey', y=-0.01)
tuftefy(ax)

In [ ]:
fig = plt.figure()
fig.set_size_inches(8, 6)

ax = sns.lineplot(data=merged_opnames[merged_opnames.Group.isin(["0-14", "15-29"])], x="Date_of_statistics_week_start", y="Hospital_admission_notification_7d", hue="Group")
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
ax.set_xlabel('')
ax.set_ylabel('Opnames')
ax.set_title('Voortschrijdend gemiddelde (35d) van opnames <30', pad=30)
plt.axvline(date(2021,12,20), ls=":", c="#999933")
#plt.axvline(date(2021,2,8), ls=":", c="#aa4499")
plt.title("bron: RIVM update " + merged_opnames.Date_of_statistics_week_start.max().strftime('%Y-%m-%d'), loc='right', fontsize=12, color='grey', y=-0.00)
tuftefy(ax)

In [ ]:
fig = plt.figure()
fig.set_size_inches(15, 10)
ax = sns.lineplot(data=merged_opnames2[merged_opnames2.Group=="30+"], color='#004488', x="Date_of_statistics_week_start", y="Hospital_admission_notification_7d", label="30+")
plt.legend(bbox_to_anchor=(0.2, 1),borderaxespad=0)
ax2 = ax.twinx()
sns.lineplot(data=merged_opnames2[merged_opnames2.Group=="<30"], color='#ddaa33', x="Date_of_statistics_week_start", y="Hospital_admission_notification_7d", label="< 30", ax=ax2)
plt.legend(bbox_to_anchor=(0.9, 1),borderaxespad=0)
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
ax.set_xlabel('')
ax.set_ylabel('Opnames 30+')
ax2.set_ylabel('Opnames < 30')
ax.set_title('Voortschrijdend gemiddelde (35d) van opnames', pad=30)
plt.title("bron: RIVM update " + merged_opnames2.Date_of_statistics_week_start.max().strftime('%Y-%m-%d'), loc='right', fontsize=12, color='grey', y=-0.00)
tuftefy(ax)
tuftefy(ax2)


## Plots NICE data

In [ ]:

fig = plt.figure()
fig.set_size_inches(15, 10)
ax = sns.lineplot(data=opnames_nice, color='#004488', x="Datum", y="30+", label="30+")
plt.legend(bbox_to_anchor=(0.2, 1),borderaxespad=0)
ax2 = ax.twinx()
sns.lineplot(data=opnames_nice, color='#ddaa33', x="Datum", y="<30", label="< 30", ax=ax2)
plt.legend(bbox_to_anchor=(0.9, 1),borderaxespad=0)
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
ax.set_xlabel('')
ax.set_ylabel('Opnames 30+')
ax2.set_ylabel('Opnames < 30')
ax.set_title('Voortschrijdend gemiddelde (35d) van opnames', pad=30)
plt.title("bron: NICE " + opnames_nice.Datum.max().strftime('%Y-%m-%d'), loc='right', fontsize=12, color='grey', y=-0.00)
tuftefy(ax)
tuftefy(ax2)

In [ ]:

fig = plt.figure()
fig.set_size_inches(15, 10)
ax = sns.lineplot(data=opnames_nice, color='#228833', x="Datum", y="60+", label="60+")
plt.legend(bbox_to_anchor=(0.2, 1),borderaxespad=0)
ax2 = ax.twinx()
sns.lineplot(data=opnames_nice, color='#4477AA', x="Datum", y="30 - 60", label="30 - 60", ax = ax2)
sns.lineplot(data=opnames_nice, color='#66CCEE', x="Datum", y="<30", label="< 30", ax=ax2)
plt.legend(bbox_to_anchor=(0.9, 1),borderaxespad=0)
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
ax.set_xlabel('')
ax.set_ylabel('Opnames 60+')
ax2.set_ylabel('Opnames < 60')
ax.set_title('Voortschrijdend gemiddelde (35d) van nieuwe opnames', pad=30)
plt.title("bron: NICE " + opnames_nice.Datum.max().strftime('%Y-%m-%d'), loc='right', fontsize=12, color='grey', y=-0.00)
tuftefy(ax)
tuftefy(ax2)

In [ ]:

fig = plt.figure()
fig.set_size_inches(15, 10)
ax = sns.lineplot(data=opnames_nice7, color='#228833', x="Datum", y="60+", label="60+")
plt.legend(bbox_to_anchor=(0.2, 1),borderaxespad=0)
ax2 = ax.twinx()
sns.lineplot(data=opnames_nice7, color='#4477AA', x="Datum", y="30 - 60", label="30 - 60", ax = ax2)
sns.lineplot(data=opnames_nice7, color='#66CCEE', x="Datum", y="<30", label="< 30", ax=ax2)
plt.legend(bbox_to_anchor=(0.9, 1),borderaxespad=0)
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
ax.set_xlabel('')
ax.set_ylabel('Opnames 60+')
ax2.set_ylabel('Opnames < 60')
ax.set_title('Voortschrijdend gemiddelde (14d) van nieuwe opnames', pad=30)
plt.title("bron: NICE " + opnames_nice.Datum.max().strftime('%Y-%m-%d'), loc='right', fontsize=12, color='grey', y=-0.00)
tuftefy(ax)
tuftefy(ax2)

In [ ]:
fig = plt.figure()
fig.set_size_inches(15, 10)
ax = sns.lineplot(data=opnames_nice, color='#228833', x="Datum", y="60+", label="60+")
sns.lineplot(data=opnames_nice, color='#4477AA', x="Datum", y="30 - 60", label="30 - 60")
sns.lineplot(data=opnames_nice, color='#66CCEE', x="Datum", y="<30", label="< 30")
plt.legend(bbox_to_anchor=(0.2, 1),borderaxespad=0)
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
ax.set_title('Voortschrijdend gemiddelde (35d) van nieuwe opnames', pad=30)
ax.set_xlabel('')
ax.set_ylabel('Opnames')
plt.title("bron: NICE " + opnames_nice.Datum.max().strftime('%Y-%m-%d'), loc='right', fontsize=12, color='grey', y=-0.00)
tuftefy(ax)

In [ ]:
fig = plt.figure()
fig.set_size_inches(15, 10)
ax = sns.lineplot(data=opnames_nice7, color='#228833', x="Datum", y="60+", label="60+")
sns.lineplot(data=opnames_nice7, color='#4477AA', x="Datum", y="30 - 60", label="30 - 60")
sns.lineplot(data=opnames_nice7, color='#66CCEE', x="Datum", y="<30", label="< 30")
plt.legend(bbox_to_anchor=(0.2, 1),borderaxespad=0)
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
ax.set_title('Voortschrijdend gemiddelde (14d) van nieuwe opnames', pad=30)
ax.set_xlabel('')
ax.set_ylabel('Opnames')
plt.title("bron: NICE " + opnames_nice.Datum.max().strftime('%Y-%m-%d'), loc='right', fontsize=12, color='grey', y=-0.00)
tuftefy(ax)

In [ ]:
sns.set_context("talk", font_scale=0.9, rc={"lines.linewidth": 3, "font.size":8,"axes.titlesize":24,"axes.labelsize":18})
sns.set_style("whitegrid")
sns.set_theme(style="ticks")
#sns.set_palette(sns.color_palette("colorblind"))
sns.set_palette(sns.color_palette(['#CC6677', '#332288', '#DDCC77', '#117733', '#88CCEE', \
                    '#882255', '#44AA99', '#999933', '#AA4499', '#DDDDDD', \
                    '#000000']))
                    
g = sns.FacetGrid(data=opnames_cases_joined[opnames_cases_joined.Date> '2021-08-01'], col="Age_group", col_wrap=3, margin_titles=True, legend_out=True, sharey=False)
g.map_dataframe(sns.lineplot, x='Date', y='PositieveTestGrowth', color='#CC6677', label='Positieve tests')
g = g.map(plt.axhline, y=0.0, ls="-", c=".8", zorder=1)
g.map_dataframe(sns.lineplot, x='Date',y='OpnameGrowth', color='#332288', label='Opnames')
g.map_dataframe(sns.lineplot, x='Date',y='ICGrowth', color='#DDCC77', label='IC')
g.set_axis_labels("", "Groei percentage")

for ax in g.axes:
  #ax.xaxis.set_major_formatter(mdates.DateFormatter('%V'))
  #ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MONDAY, interval=1))
  ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))

g.set_titles(col_template = '{col_name}')
g.add_legend(loc='upper right')

g.despine()
plt.tight_layout()
plt.subplots_adjust(top=0.94)

In [ ]:
def facet_age(groups):
    g = sns.FacetGrid(data=opnames_cases_joined[(opnames_cases_joined.Date> '2021-05-01') & opnames_cases_joined.Age_group.isin(groups)], col="Age_group", col_wrap=min(3, len(groups)), margin_titles=True, legend_out=True, sharey=False)
    g.map_dataframe(sns.lineplot, x='Date', y='PositieveTestGrowth', color='#CC6677', label='Positieve tests')
    g = g.map(plt.axhline, y=0.0, ls="-", c=".8", zorder=1)
    g.map_dataframe(sns.lineplot, x='Date',y='OpnameGrowth', color='#332288', label='Opnames')
    g.map_dataframe(sns.lineplot, x='Date',y='ICGrowth', color='#DDCC77', label='IC')
    g.set_axis_labels("", "Groei percentage")

    for ax in g.axes:
        #ax.xaxis.set_major_formatter(mdates.DateFormatter('%V'))
        #ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MONDAY, interval=1))
        ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))

    g.set_titles(col_template = '{col_name}')
    g.add_legend(loc='upper right')

    g.despine()
    plt.tight_layout()
    plt.subplots_adjust(top=0.94)
    plt.show()
    plt.close()


facet_age(["<30", "30 - 60", "60+"])
facet_age(["<30", "30+"])
facet_age(["0-9", "10-19", '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90+'])

    